In [2]:
!pip install potrace

ERROR: Could not find a version that satisfies the requirement potrace (from versions: none)
ERROR: No matching distribution found for potrace


In [5]:
!pip install pillow opencv-python svgwrite

   ---------------------------------------- 0.0/67.1 kB ? eta -:--:--
   ------ --------------------------------- 10.2/67.1 kB ? eta -:--:--
   ---------------------------------------- 67.1/67.1 kB 902.9 kB/s eta 0:00:00


In [9]:
import cv2
import svgwrite
from PIL import Image

def raster_to_svg(image_path, output_svg_path):
    # Load the image using PIL and convert it to grayscale
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image.save("temp_grayscale.png")  # Save temporarily for OpenCV to read

    # Read the image using OpenCV
    img = cv2.imread("temp_grayscale.png", 0)

    # Threshold the image to get a binary version
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an SVG file to write the contours as paths
    dwg = svgwrite.Drawing(output_svg_path, profile='tiny')

    # Iterate through each contour and convert it into an SVG path
    for contour in contours:
        path_data = "M " + " ".join(f"{point[0][0]},{point[0][1]}" for point in contour)
        path_data += " Z"
        dwg.add(dwg.path(d=path_data, fill='black'))

    # Save the SVG file
    dwg.save()
    print(f"SVG vector graphic saved to {output_svg_path}")

# Example usage
image_path = "dfsfs.png"  # Replace with your image path
output_svg_path = "output_vector_image.svg"  # Path to save the SVG file

raster_to_svg(image_path, output_svg_path)


SVG vector graphic saved to output_vector_image.svg


In [8]:
from PIL import Image
import potrace

def raster_to_svg(image_path, output_svg_path):
    # Open the image using PIL
    image = Image.open(image_path).convert('L')  # Convert to grayscale for tracing

    # Create a bitmap from the grayscale image
    bitmap = potrace.Bitmap(image)
    path = bitmap.trace()

    # Create the SVG file and write the vector paths
    with open(output_svg_path, 'w') as svg_file:
        svg_file.write('<svg xmlns="http://www.w3.org/2000/svg" version="1.1">\n')

        # Iterate through the paths and write them as SVG paths
        for curve in path:
            svg_file.write('<path d="')
            for segment in curve:
                if segment.is_corner:
                    svg_file.write(f'L {segment.c1.x} {segment.c1.y} ')
                else:
                    svg_file.write(f'Q {segment.c1.x} {segment.c1.y} {segment.c2.x} {segment.c2.y} ')
            svg_file.write(f'Z" fill="black"/>\n')
        
        svg_file.write('</svg>\n')

    print(f"SVG vector graphic saved to {output_svg_path}")

# Example usage
image_path = "dfsfs.png"  # Replace with your image path
output_svg_path = "output_vector_image.svg"  # Path to save the SVG file

raster_to_svg(image_path, output_svg_path)


ModuleNotFoundError: No module named 'potrace'

In [10]:
import cv2
import svgwrite
from PIL import Image
import os

def raster_to_svg(image_path, output_svg_path):
    # Check if the input image exists
    if not os.path.exists(image_path):
        print(f"Error: The file '{image_path}' does not exist.")
        return

    try:
        # Load the image using PIL and convert it to grayscale
        image = Image.open(image_path).convert('L')  # Convert to grayscale
        temp_image_path = "temp_grayscale.png"
        image.save(temp_image_path)  # Save temporarily for OpenCV to read

        # Read the image using OpenCV
        img = cv2.imread(temp_image_path, 0)

        # Use adaptive thresholding for better results
        binary = cv2.adaptiveThreshold(
            img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
        )

        # Find contours in the binary image
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create an SVG file to write the contours as paths
        dwg = svgwrite.Drawing(output_svg_path, profile='tiny')

        # Iterate through each contour and convert it into an SVG path
        for contour in contours:
            path_data = "M " + " ".join(f"{point[0][0]},{point[0][1]}" for point in contour)
            path_data += " Z"
            dwg.add(dwg.path(d=path_data, fill='black'))

        # Save the SVG file
        dwg.save()
        print(f"SVG vector graphic saved to {output_svg_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
image_path = "dfsfs.png"  # Replace with your image path
output_svg_path = "output_vector_image.svg"  # Path to save the SVG file

raster_to_svg(image_path, output_svg_path)


SVG vector graphic saved to output_vector_image.svg


In [11]:
import cv2
import svgwrite
from PIL import Image
import numpy as np
import os

def raster_to_color_svg(image_path, output_svg_path):
    # Check if the input image exists
    if not os.path.exists(image_path):
        print(f"Error: The file '{image_path}' does not exist.")
        return

    try:
        # Load the image using OpenCV
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Failed to load the image from '{image_path}'")
            return

        # Convert to RGB format (OpenCV loads images as BGR by default)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Resize the image if needed for better processing
        scale_percent = 50  # Reduce size to 50% for faster processing (adjust as needed)
        width = int(img_rgb.shape[1] * scale_percent / 100)
        height = int(img_rgb.shape[0] * scale_percent / 100)
        resized_img = cv2.resize(img_rgb, (width, height), interpolation=cv2.INTER_AREA)

        # Convert the resized image to a grayscale image for contour detection
        gray = cv2.cvtColor(resized_img, cv2.COLOR_RGB2GRAY)
        _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

        # Find contours in the binary image
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create an SVG file to write the contours as paths with color fills
        dwg = svgwrite.Drawing(output_svg_path, profile='tiny', size=(width, height))

        # Iterate through each contour
        for contour in contours:
            # Create a mask for the current contour
            mask = np.zeros_like(resized_img)
            cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

            # Calculate the mean color inside the contour
            mean_color = cv2.mean(resized_img, mask=cv2.cvtColor(mask, cv2.COLOR_RGB2GRAY))[:3]
            fill_color = svgwrite.utils.rgb(*[int(c) for c in mean_color])

            # Create the SVG path data for the contour
            path_data = "M " + " ".join(f"{point[0][0]},{point[0][1]}" for point in contour)
            path_data += " Z"

            # Add the path to the SVG with the calculated fill color
            dwg.add(dwg.path(d=path_data, fill=fill_color, stroke='none'))

        # Save the SVG file
        dwg.save()
        print(f"SVG vector graphic saved to {output_svg_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
image_path = "dfsfs.png"  # Replace with your image path
output_svg_path = "output_color_vector_image.svg"  # Path to save the SVG file

raster_to_color_svg(image_path, output_svg_path)


SVG vector graphic saved to output_color_vector_image.svg
